## Import The Required Libraries

In [1]:
# basics
import sys
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import cv2
import pickle 

import shutil
# Keras Libraries
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l2
from keras.models import Sequential , Functional
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.utils import  load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report, roc_auc_score
import keras.backend
import os
import random
import time
import datetime
from tensorflow.python.framework import ops
import inspect
import gc

Mounted at /content/gdrive


## utility functions

In [27]:
# print date and time for given type of representation
def date_time(x):
    if x==1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==2:    
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==3:  
        return 'Date now: %s' % datetime.datetime.now()
    if x==4:  
        return 'Date today: %s' % datetime.date.today() 

## code


In [28]:
batch_size =16
model_name="vgg16"
fold_name="fold0"
input_directory = "/content/gdrive/MyDrive/Dataset/data_{}/".format(fold_name)
output_directory = "/content/gdrive/MyDrive/Dataset/output/"

train_path = input_directory + r"train"
val_path = input_directory + r"valid"
test_path = input_directory + r"test"

figure_directory = "output/figures" 


### preprosessing

In [29]:

seed_value= 42

os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)

np.random.seed(seed_value)

#tf.set_random_seed(seed_value)
tf.random.set_seed(seed_value)


session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [30]:
datagen = ImageDataGenerator( rescale=1.0/255.0,
                                   rotation_range=5,
    
    width_shift_range=0.01,
    height_shift_range=0.01)



train_generator = datagen.flow_from_directory(directory = train_path, 
                                                    color_mode="rgb",
                                                    target_size = (240, 240), # image height , image width
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=42)

test_generator = datagen.flow_from_directory(directory = test_path, 
                                                 color_mode="rgb",
                                                    target_size = (240, 240), # image height , image width
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=42)

test_generator.reset()

Found 146 images belonging to 4 classes.
Found 98 images belonging to 4 classes.


In [31]:
x,y = next(train_generator)
x.shape

(16, 240, 240, 3)

# Fine tuning

In [32]:
# Load pre-trained model
base_model= tf.keras.models.load_model('/content/gdrive/MyDrive/Dataset/output/models/Model_frame_{}.h5'.format(model_name))
model = Sequential()

for layer in base_model.layers[:-1]: 
    model.add(layer)  

# Freeze the layers of the base model
for layer in model.layers:
    layer.trainable = True
base_model=model

# add new layers to the base model
x = base_model.output
#x = keras.layers.Flatten()(x)
#x = keras.layers.Dense(1024, activation='relu')(x)
#x = keras.layers.Dense(256, activation='relu')(x)
#x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(4, activation='softmax')(x)

# create a new model with the new layers
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

In [33]:
early = tf.keras.callbacks.EarlyStopping( patience=10,
                                          min_delta=0.001,
                                          restore_best_weights=True,
                                         #monitor='loss', 
                                         #mode='min'
)
checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath=r"/content/gdrive/MyDrive/Dataset/output/checkpoints/model_tuned_"+model_name+"_"+fold_name+"_{epoch}.h5",
   save_best_only=True,
    verbose=1,
    #monitor='loss', 
    #mode='min'

)
reduce1=tf.keras.callbacks.ReduceLROnPlateau(patience=10,
                                             #monitor='loss', 
                                             #mode='min',  # after 10 epochs with no improvement
    verbose=1)
callbacks=[early,checkpoint,reduce1]
model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=.001),loss='categorical_crossentropy',metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16_input (InputLayer)    [(None, 240, 240, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        3584      
 ormalization)                                                   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              525312    
                                                                 
 dense_1 (Dense)             (None, 512)               5248

In [34]:
#model=tf.keras.models.load_model("/content/gdrive/MyDrive/Dataset/output/checkpoints/model_tuned_vgg16_fold3_19.h5")


In [ ]:
epochs=25
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = test_generator.n//test_generator.batch_size

print("start Model Trainning", date_time(1))
# fit model
history = model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,
                    callbacks=callbacks)


print("Completed Model Trainning", date_time(1))

In [ ]:
model.save("/content/gdrive/MyDrive/Dataset/output/tmp/models/model_tuned_"+model_name+"_{}.h5".format(fold_name))

## Visualization

In [ ]:
def create_charts(cnn, cnn_model):
    ## Define train & validation loss
    train_loss = cnn_model.history['loss']
    val_loss = cnn_model.history['val_loss']
    #print(list(cnn_model.history.keys()))
    ## Define train & validation AUC
    train_auc_name = list(cnn_model.history.keys())[1]
    val_auc_name = list(cnn_model.history.keys())[3]
    train_auc = cnn_model.history[train_auc_name]
    val_auc = cnn_model.history[val_auc_name]
    
    ## Define y_pred & y_true
    y_true = test_generator.classes
    
    Y_pred = cnn.predict(test_generator, steps = len(test_generator))
    print(y_true)
    print(Y_pred)
   
   
    #y_pred = (Y_pred > 0.5).T[0]
    y_pred = np.argmax(Y_pred, axis=1)
    print(y_pred)
    ## PLOT ##
    fig = plt.figure(figsize=(13, 10))
    
    ## PLOT 1: TRAIN VS. VALIDATION LOSS 
    plt.subplot(2,2,1)
    plt.title("Training vs. Validation Loss")
    plt.plot(train_loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()

    ## PLOT 2: TRAIN VS. VALIDATION ACC
    plt.subplot(2,2,2)
    plt.title("Training vs. Validation ACC Score")
    plt.plot(train_auc, label='training acc')
    plt.plot(val_auc, label='validation acc')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()
    
    ## PLOT 3: CONFUSION MATRIX
    plt.subplot(2,2,3)
      # Set up the labels for in the confusion matrix
    cm = confusion_matrix(y_true, y_pred,labels=[0,1,2,3])
    names = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    #print(counts)
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [[f'{names[i]}\n{percentages[i]}' for i in range(len(names))] for j in range(len(cm))]
    #print(labels)
    #labels = np.asarray(labels).reshape(2,2)
    ticklabels = ['socre0', 'socre1','socre2','socre3']
    #print(cm)
    #print(labels)
    # Create confusion matrix as heatmap
    sns.set(font_scale = 1.4)
    ax = sns.heatmap(cm, annot=True, fmt='', cmap='Oranges', xticklabels=ticklabels, yticklabels=ticklabels )
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.title("Confusion Matrix") #plt.title("Confusion Matrix\n", fontsize=10)
    plt.xlabel("Predicted", size=14)
    plt.ylabel("Actual", size=14) 
    plt.savefig("/content/gdrive/MyDrive/Dataset/output/figures/cm_model_tuned_"+model_name+"_{}.jpg".format(fold_name))
    
  
    plt.xlabel('False Positive Rate', size=14)
    plt.ylabel('True Positive Rate', size=14)
    plt.legend(loc='best')
    #plt.savefig('roc.png', bbox_inches='tight', pad_inches=1)
    
    ## END PLOTS
    plt.tight_layout();
    
    ## Summary Statistics
    
    accuracy = np.trace(cm) / np.sum(cm)
    precision = np.diag(cm) / np.sum(cm, axis = 0)
    precision = np.mean(precision)  
    recall = np.diag(cm) / np.sum(cm, axis = 1)
    recall = np.mean(recall)
    f1 = 2 * (precision * recall) / (precision + recall)
    TN = np.sum(np.diag(cm)) - np.sum(cm)
    FP = np.sum(cm, axis=0) - np.diag(cm)
    specificity = TN / (TN+FP) # % negative out of all supposed to be negatives
    specificity = np.mean(specificity)
    
    stats_summary = '[Summary Statistics]\nAccuracy = {:.2%} | Precision = {:.2%} | Recall = {:.2%} | Specificity = {:.2%} | F1 Score = {:.2%}'.format(accuracy, precision, recall, specificity, f1)
    print(stats_summary)

In [ ]:
create_charts(model, history)

In [ ]:
accuracy_score = model.evaluate(test_generator)
print(accuracy_score)
print("Accuracy: {:.4f}%".format(accuracy_score[1] * 100))
 
print("Loss: ",accuracy_score[0])